In [101]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt


In [102]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, LancasterStemmer, PorterStemmer
from nltk import word_tokenize, pos_tag
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


pd.set_option('display.max_columns', None)

from gensim.models import Word2Vec


from matplotlib import pyplot as plt
import seaborn as sns


from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Use fivethirtyeight style
plt.style.use('fivethirtyeight')

#Cancel any style and reset matplotlib styles
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]


In [104]:
df = pd.read_csv("C:/Users/dimit/OneDrive/Desktop/git_projects/Recommendation System/Recommendation-System/n_movies.csv")
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601"


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


In [106]:
df = df.drop_duplicates()

In [107]:
df.isnull().sum()

title             0
year            527
certificate    3453
duration       2036
genre            73
rating         1173
description       0
stars             0
votes          1173
dtype: int64

In [108]:
words = stopwords.words("english")

In [109]:
lemmatizer = WordNetLemmatizer()

In [110]:
df['description'] = df['description'].apply(lambda x: " ".join([lemmatizer.lemmatize(i) for i in re.sub("[^a-zA-Z]", " ", str(x)).split() if i not in words]).lower())

In [111]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [112]:

nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [113]:
df['tokenized'] = df['description'].apply(word_tokenize)
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,tokenized
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031","[decades, all, valley, karate, tournament, bou..."
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885","[follows, political, rivalry, romance, queen, ..."
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384","[the, trial, tribulation, criminal, lawyer, ji..."
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773","[when, psychiatrist, shelter, mysterious, cult..."
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413","[a, street, kid, trying, survive, technology, ..."


In [114]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [115]:
df['POS'] = df['tokenized'].apply(pos_tag)
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,tokenized,POS
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031","[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka..."
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885","[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN..."
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384","[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c..."
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773","[when, psychiatrist, shelter, mysterious, cult...","[(when, WRB), (psychiatrist, NN), (shelter, RB..."
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413","[a, street, kid, trying, survive, technology, ...","[(a, DT), (street, NN), (kid, NN), (trying, VB..."


In [116]:
tag_filter = ["NN","NNS","NNP","NNPS"]  #keep only nouns, adjectives and adverbs
# tag_filter = ["NN","NNS","NNP","NNPS","RB","RBR","RBS","JJ","JJR","JJS"] 
# filter pos function
def pos_filter(row):
    keep_list = []
    for x,y in row:
        if y in tag_filter:
            keep_list.append(x)
    return keep_list
df['clean'] = df['POS'].apply(pos_filter)
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,tokenized,POS,clean
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031","[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[decades, tournament, daniel, larusso, johnny,..."
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885","[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[rivalry, romance, reign, event, half, century]"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384","[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[trial, tribulation, lawyer, jimmy, mcgill, ru..."
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773","[when, psychiatrist, shelter, mysterious, cult...","[(when, WRB), (psychiatrist, NN), (shelter, RB...","[psychiatrist, cult, escapee, world, girl, arr..."
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413","[a, street, kid, trying, survive, technology, ...","[(a, DT), (street, NN), (kid, NN), (trying, VB...","[street, kid, technology, body, modification, ..."


In [117]:
def put_title_first(row):
    title = row['title']
    description = row['clean']
    if isinstance(description, list):
        description.insert(0, title)
    else:
        description = [title, description]
    return description



def put_genre_second(row):
    genre = row['genre']
    description = row['clean']
    if isinstance(description, list):
        description.insert(1, genre)
    else:
        description = [genre, description]
    return description

In [118]:
df['movie_description'] = df.apply(put_title_first, axis=1)

In [119]:
df['movie_description'] = df.apply(put_genre_second, axis=1)

In [120]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes,tokenized,POS,clean,movie_description
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031","[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[Cobra Kai, Action, Comedy, Drama, decades, to...","[Cobra Kai, Action, Comedy, Drama, decades, to..."
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885","[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[The Crown, Biography, Drama, History, rivalry...","[The Crown, Biography, Drama, History, rivalry..."
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384","[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[Better Call Saul, Crime, Drama, trial, tribul...","[Better Call Saul, Crime, Drama, trial, tribul..."
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773","[when, psychiatrist, shelter, mysterious, cult...","[(when, WRB), (psychiatrist, NN), (shelter, RB...","[Devil in Ohio, Drama, Horror, Mystery, psychi...","[Devil in Ohio, Drama, Horror, Mystery, psychi..."
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413","[a, street, kid, trying, survive, technology, ...","[(a, DT), (street, NN), (kid, NN), (trying, VB...","[Cyberpunk: Edgerunners, Animation, Action, Ad...","[Cyberpunk: Edgerunners, Animation, Action, Ad..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,after experimental gene therapy turn monster t...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130","[after, experimental, gene, therapy, turn, mon...","[(after, IN), (experimental, JJ), (gene, NN), ...","[The Imperfects, Action, Adventure, Drama, gen...","[The Imperfects, Action, Adventure, Drama, gen..."
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,sheriff deputy rick grimes wake coma learn wor...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067","[sheriff, deputy, rick, grimes, wake, coma, le...","[(sheriff, JJ), (deputy, NN), (rick, NN), (gri...","[The Walking Dead, Drama, Horror, Thriller, de...","[The Walking Dead, Drama, Horror, Thriller, de..."
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898","[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[The Crown, Biography, Drama, History, rivalry...","[The Crown, Biography, Drama, History, rivalry..."
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,two brother follow father footstep hunter figh...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601","[two, brother, follow, father, footstep, hunte...","[(two, CD), (brother, NN), (follow, VBP), (fat...","[Supernatural, Drama, Fantasy, Horror, brother...","[Supernatural, Drama, Fantasy, Horror, brother..."


In [121]:
list_of_list = df['movie_description'].tolist()

In [122]:
list_of_list

[['Cobra Kai',
  'Action, Comedy, Drama',
  'decades',
  'tournament',
  'daniel',
  'larusso',
  'johnny',
  'lawrence',
  'art',
  'rival'],
 ['The Crown',
  'Biography, Drama, History',
  'rivalry',
  'romance',
  'reign',
  'event',
  'half',
  'century'],
 ['Better Call Saul',
  'Crime, Drama',
  'trial',
  'tribulation',
  'lawyer',
  'jimmy',
  'mcgill',
  'run',
  'jesse',
  'pinkman'],
 ['Devil in Ohio',
  'Drama, Horror, Mystery',
  'psychiatrist',
  'cult',
  'escapee',
  'world',
  'girl',
  'arrival',
  'family'],
 ['Cyberpunk: Edgerunners',
  'Animation, Action, Adventure',
  'street',
  'kid',
  'technology',
  'body',
  'modification',
  'city',
  'future',
  'everything',
  'chooses',
  'edgerunner',
  'outlaw',
  'cyberpunk'],
 ['The Sandman',
  'Drama, Fantasy, Horror',
  'decade',
  'imprisonment',
  'wizard',
  'dream',
  'personification',
  'dream',
  'reclaim',
  'equipment'],
 ['Rick and Morty',
  'Animation, Adventure, Comedy',
  'series',
  'scientist',
  'gr

In [123]:
model = Word2Vec(list_of_list, vector_size=50, window=5, min_count=5, workers=4, sample=1e-3, sg=1, negative=5)

In [124]:
model.wv.most_similar(['The Sandman'],topn=20)


[('Crime, Drama, Horror', 0.9958388209342957),
 ('harry', 0.9955827593803406),
 ('none', 0.9955184459686279),
 ('colette', 0.9953740239143372),
 ('catherine', 0.9953222274780273),
 ('chase', 0.9952656626701355),
 ('deed', 0.9952526092529297),
 ('researcher', 0.9951983690261841),
 ('dan', 0.9950603246688843),
 ('score', 0.9950516819953918),
 ('jessica', 0.9950322508811951),
 ('delivery', 0.9949995875358582),
 ('marcus', 0.9949971437454224),
 ('vehicle', 0.9949859976768494),
 ('push', 0.9949113726615906),
 ('asia', 0.994910478591919),
 ('facility', 0.9949032068252563),
 ('boleyn', 0.9949003458023071),
 ('demand', 0.9948863387107849),
 ('realization', 0.994881272315979)]

In [129]:
model.wv.similarity('The Sandman', 'Rick and Morty')

0.98759556

In [128]:
for t in df['title']:
    try:
        print(t)
        print('==================')
        print(model.wv[t])
    except:
        pass

Cobra Kai
[-0.10344768  0.02141722 -0.12700818 -0.05780277 -0.379093   -0.12512387
  0.12485425  0.4942615  -0.23367289 -0.14881176  0.08127704 -0.25951475
  0.05099856  0.16714032  0.02156716  0.10038434  0.11701746  0.05634686
 -0.20373227  0.05665348  0.05122628  0.15785065  0.35904503 -0.17789909
  0.16981299 -0.03912739 -0.21014564 -0.18748048 -0.15772615  0.06816527
  0.18485574 -0.056858   -0.20693749 -0.06414568 -0.02632047  0.24152677
  0.0815503   0.28146455  0.03580144 -0.08432516  0.39772886 -0.12250149
 -0.13479559  0.12649435  0.3613672  -0.0645062  -0.01988776 -0.10332514
  0.1177545   0.05567611]
The Crown
Better Call Saul
Devil in Ohio
Cyberpunk: Edgerunners
The Sandman
[-0.09494423  0.0289753  -0.12901974 -0.06565283 -0.31084818 -0.11271892
  0.13164377  0.45017132 -0.16515435 -0.1628511   0.05490841 -0.19991766
  0.0445899   0.14088763  0.02313811  0.07410068  0.11172998  0.05354387
 -0.19670463  0.09503888  0.02381841  0.16731758  0.30632973 -0.17230628
  0.1537568 